In [0]:
!pip install pytorch_pretrained_bert


     |████████████████████████████████| 133kB 2.7MB/s 
     |████████████████████████████████| 645kB 67.8MB/s 


In [0]:
!pip install transformers

     |████████████████████████████████| 368kB 2.8MB/s 
     |████████████████████████████████| 860kB 52.6MB/s 
     |████████████████████████████████| 1.0MB 44.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=a4111da7d9693a65a2bc683e1d07bc4510a80c804af872380861d82fbe21b520
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import torch
import numpy as np
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from transformers import  BertModel
from pytorch_pretrained_bert import BertConfig, BertForPreTraining

In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': ''})
download.GetContentFile('DOWNLOAD.tar')

In [0]:
download = drive.CreateFile({'id': ''})
download.GetContentFile('train_qa.csv')

In [0]:
!tar -xvf  'DOWNLOAD.tar' 'bert_model.ckpt'

bert_model.ckpt


In [0]:
TORCH_MODEL_PATH = 'bert_model.ckpt'

In [0]:
bert_dict = torch.load(TORCH_MODEL_PATH)

In [0]:
from google.colab import files
uploaded = files.upload()

Saving bert_config.json to bert_config.json
Saving vocab.txt to vocab.txt


In [0]:
!mv bert_config.json config.json

In [0]:
!mv bert_model.ckpt pytorch_model.bin

In [0]:
CONFIG_FILE = 'bert_config.json'

#config = BertConfig.from_json_file(CONFIG_FILE)
bert = BertModel.from_pretrained('.', from_tf=False)
#bert.load_state_dict(bert_dict)

In [0]:
bert.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [0]:
from torch import nn
from torch.nn import CrossEntropyLoss

In [0]:
import torch.nn.functional as F

class BertForQuestionAnswering(nn.Module):
    
    def __init__(self, bertModel):
        super().__init__()
        self.num_labels = 2

        self.bert = bertModel#BertModel.from_pretrained('bert-base-multilingual-cased')
        self.bert.eval()
        self.qa_outputs = nn.Sequential(nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(64, 2))
        self.loss_fct = CrossEntropyLoss()
        
    

    def forward(self, input_ids=None, token_type_ids=None, start_positions=None, end_positions=None, mask=None):
        output = self.bert(input_ids, attention_mask=mask)

        sequence_output = output[0]

        logits = self.qa_outputs(sequence_output)
        loss = None

        if start_positions is not None and end_positions is not None:
            loss = (self.loss_fct(logits[:, :, 0].masked_fill((1 - mask).bool(), float('-inf')), start_positions) + \
                   self.loss_fct(logits[:, :, 1].masked_fill((1 - mask).bool(), float('-inf')), end_positions)) / 2

        return loss, F.softmax(logits.masked_fill((1 - mask[:, :, None]).bool(), float('-inf')), dim=1)

In [0]:
from transformers import BertTokenizer

In [0]:
tokenizer = BertTokenizer.from_pretrained('.', cache_dir=None, do_lower_case=False)
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [0]:
def preprocess(text, question, ans):
    answer = answer.lower()
    if answer not in text.lower():
        return [], []
    
    firstInText = text.lower().find(answer)
    lastInText = first + len(answer)
    text_tokens = tokenizer.tokenize(text[:firstInText].strip())
    first = len(text_tokens)
    text_tokens += tokenizer.tokenize(text[firstInText:lastInText].strip())
    last = len(text_tokens) - 1
    text_tokens += tokenizer.tokenize(text[lastInText:].strip())
    question_tokens = tokenizer.tokenize(question)
    
    length = MAX_TEXT_LEN - len(question_tokens) - 3
    if len(text_tokens) > length:
        part_length = length // 3
        stride = 3 * part_length
        nrow = np.ceil(len(text_tokens) / part_length) - 2
        indexes = part_length * np.arange(nrow)[:, None] + np.arange(stride)
        print(indexes)
        indexes = indexes.astype(np.int32)

        max_index = indexes.max()
        diff = max_index + 1 - len(text_tokens)
        text_tokens += diff * [tokenizer.pad_token]

        text_tokens = list(np.array(text_tokens)[indexes])
        
        tokens = []
        labels = []
        for i, ts in enumerate(text_tokens):
            while ts[-1] == tokenizer.pad_token:
                ts = ts[:-1]
                
            tokens += [ts]
                
            lfirst = first - i * part_length
            llast = last - i * part_length
            
            mask = lfirst >= 0 and lfirst < len(ts) and llast >= 0 and llast < len(ts)
            labels += [(lfirst if mask else 0, llast if mask else 0)]
    else:
        tokens = [text_tokens]
        labels = [(first, last)]
        
    for i in range(len(tokens)):
        tokens[i] = [tokenizer.cls_token] + \
                    question_tokens + \
                    [tokenizer.sep_token] + \
                    tokens[i] + \
                    [tokenizer.sep_token]
        labels[i] = ((labels[i][0][0] + 2 + len(question_tokens), labels[i][0][1]),
                     (labels[i][1][0] + 2 + len(question_tokens), labels[i][1][1]))

    return tokens, labels

In [0]:
def pad_sequence(texts):
    max_len = max([len(text) for text in texts])
    masks = [[1] * len(text) + [0] * (max_len - len(text)) for text in texts]
    texts = [text + [tokenizer.pad_token] * (max_len - len(text)) for text in texts]
    texts = [tokenizer.convert_tokens_to_ids(text) for text in texts]
    texts = torch.LongTensor(texts)
    masks = torch.LongTensor(masks)

    return texts, masks

def collate_fn(data):
    texts, labels = zip(*data)
    texts, masks = pad_sequence(texts)
    
    labels_first, labels_last = zip(*labels)
    start_pos = zip(*labels_first)
    end_pos = zip(*labels_last)
    
    return texts, masks, torch.LongTensor(start_pos), torch.LongTensor(end_pos)

In [0]:
MAX_TEXT_LEN = 256

In [0]:
import csv
from tqdm import tqdm

dataset_tokens, dataset_labels = [], []
with open('train_qa.csv', newline='') as csvfile:
        data = csv.reader(csvfile, delimiter=',')
        for row in tqdm(data):
            try:
                text = row[-3]
                question = row[-2]
                ans = row[-1]
                tokens, labels = preprocess(text, question, ans)
                dataset_tokens += tokens
                dataset_labels += labels
            except Exception as e:
                print(e)

50365it [01:53, 445.48it/s]


In [0]:
print(len(dataset_tokens))

50363


In [0]:
train_data_loader = torch.utils.data.DataLoader(list(zip(dataset_tokens, dataset_labels)), batch_size=16, shuffle=True,collate_fn=collate_fn)

In [0]:
!pip3 install wandb

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
     |████████████████████████████████| 256kB 63.7MB/s 
     |████████████████████████████████| 460kB 60.7MB/s 
     |████████████████████████████████| 92kB 13.2MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 184kB 67.9MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
  Created wheel for gql: filename=gql-0.1.0-cp36-none-any.whl size=5541 sha256=19b1e36448d45f6107e50e336b3dc2ffd602d62ad3a7c338be967111893f4383
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1faef575bc2a1664
  Created wheel for shortuuid: filename=shortuuid-0.5.0-cp36-none-any.whl size=5499 sha256=0bc2434aa9be1bcae1182af6e3fb6c0c654d0a75d79c8db47b63e5636e054649
  Stored in directory: /root/.cache/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576
  Created wheel for watchdog: filename=wat

In [0]:
!wandb login

In [0]:
import wandb
wandb.init(project="dul")


W&B Run: https://app.wandb.ai/vladimirrim/dul/runs/y16g9oh7

In [0]:
model = BertForQuestionAnswering(bert)

In [0]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 0.00005, weight_decay=0.000001)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.3)
epochs = 3
device = 'cuda'
model.to(device)

for epoch in range(epochs):
    model.train()
    for texts, masks, start_pos, end_pos in train_data_loader:
        optimizer.zero_grad()
        loss, _ = model(texts.to(device),
                        mask=masks.to(device),
                        start_positions=torch.tensor(start_pos).to(device),
                        end_positions=torch.tensor(end_pos).to(device))
        wandb.log({'loss' : float(loss)})
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


KeyboardInterrupt: ignored

In [0]:
torch.save(model.state_dict(), 'tmp')
uploaded = drive.CreateFile({'title': 'model'})
uploaded.SetContentFile('tmp')
uploaded.Upload()

In [0]:
model = BertForQuestionAnswering(bert)
model.load_state_dict(torch.load('tmp'))
model.eval()

In [0]:
model.cuda()

In [0]:
import torch.nn.functional as F
import re

def getBestProb(probs):
    n = len(probs)
    start, end, bestProb = 0, 0, 0
    for i in range(n):
        for j in range(i, n):
            prob = probs[i, 0] * probs[j, 1]
            if bestProb < prob:
                bestProb, start, end = prob, i, j
              
    return start, end


def concat(tokens):
    tokens = [token.replace('#', '') for token in tokens]
    return ' '.join(list(filter(lambda s: s != tokenizer.unk_token, tokens))).strip()
              


with open('output', 'w') as f:
     f.write('')


model.eval()
with torch.no_grad():
  with open('dataset_281937_1 (2).txt', newline='') as csvfile:
    test = csv.reader(csvfile, delimiter='\t')
    for row in tqdm(test):
        question_id, paragraph, question = row[-3], row[-2], row[-1]

        question_tokens = tokenizer.tokenize(question)
        text_tokens = tokenizer.tokenize(paragraph)
        
        all_tokens = [tokenizer.cls_token] + \
                     question_tokens + \
                     [tokenizer.sep_token] + \
                     text_tokens + \
                     [tokenizer.sep_token]

        length = MAX_TEXT_LEN - len(question_tokens) - 3
        if (len(text_tokens) > length):
            part_length = length // 3
            stride = 3 * part_length
            nrow = np.ceil(len(text_tokens) / part_length) - 2
            indexes = part_length * np.arange(nrow)[:, None] + np.arange(stride)
            indexes = indexes.astype(np.int32)

            max_index = indexes.max()
            diff = max_index + 1 - len(text_tokens)
            text_tokens += diff * [tokenizer.pad_token]
            text_tokens = list(np.array(text_tokens)[indexes])

            all_probs = []
            for i, ts in enumerate(text_tokens):
                while ts[-1] == tokenizer.pad_token:
                    ts = ts[:-1]

                tokens = [tokenizer.cls_token] + \
                     question_tokens + \
                     [tokenizer.sep_token] + \
                     ts + \
                     [tokenizer.sep_token]

                texts, masks = pad_sequence([tokens])
                texts = texts.to(device)
                masks = masks.to(device)

                _, probs = model(texts, mask=masks)
                probs = probs.squeeze(0)
                probs = lps[2 + len(question_tokens):]
                probs[:, 0] = F.softmax(probs[:, 0], 0)
                probs[:, 1] = F.softmax(probs[:, 1], 0)
                probs = probs.cpu().numpy()
                all_probs += list(probs)
            
            start, end = get_best(np.array(all_probs))
                
            start += 2 + len(question_tokens)
            end += 2 + len(question_tokens)
                    
        else:
            texts, masks = pad_sequence([all_tokens])
            texts = texts.to(device)
            masks = masks.to(device)

            _, probs = model(texts, mask=masks)
            probs = probs.squeeze(0)
            probs[:, 0] = F.softmax(probs[:, 0], 0)
            probs[:, 1] = F.softmax(probs[:, 1], 0)
            probs = probs.cpu().numpy()
            
            start, end = get_best(probs)
            
        s = concat(all_tokens[start: end + 1])
        with open('output', 'a') as f:
            f.write(question_id + '\t' + s + '\n')

1001it [00:23, 47.22it/s]


In [0]:
from google.colab import files
uploaded = files.upload()

Saving dataset_281937_1.txt to dataset_281937_1 (2).txt


In [0]:
uploaded = drive.CreateFile({'title': 'NLP/output'})
uploaded.SetContentFile('output')
uploaded.Upload()